In [40]:
%pylab inline
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

 # visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import ensemble, model_selection, metrics 
train_df = pd.read_csv('ready_data/train_longevity.csv')
test_df = pd.read_csv('ready_data/test_longevity.csv')
combine = [train_df, test_df]

Populating the interactive namespace from numpy and matplotlib


In [41]:
train_df.shape      # мы увидим информацию о размерности нашего датафрейма 
train_df.info()     # покажет информацию о размерности данных 
                          # описание индекса, количество not-a-number элементов 
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857 entries, 0 to 856
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Longevity      857 non-null    int64  
 1   Education      857 non-null    int64  
 2   Sex            857 non-null    object 
 3   Age            857 non-null    float64
 4   Pet            857 non-null    int64  
 5   Children       857 non-null    int64  
 6   Activity       857 non-null    int64  
 7   Sport          857 non-null    int64  
 8   Family         857 non-null    int64  
 9   IsAlone        857 non-null    int64  
 10  Age*Education  857 non-null    int64  
 11  Status         857 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 80.5+ KB


,Longevity,Education,Sex,Age,Pet,Children,Activity,Sport,Family,IsAlone,Age*Education,Status
0,0,3,female,73.0,1,0,725,1,1,0,6,3
1,1,1,male,75.0,1,0,7128,0,1,0,3,3
2,1,3,male,73.0,0,0,793,1,0,1,6,4
3,1,1,male,74.0,1,0,5310,0,1,0,2,2
4,0,3,female,74.0,0,0,805,1,0,1,6,4


fit не принимает строки, поэтому преобразуем поле SEX (0 - female, 1 - male)

In [42]:
def isMale(t):
    if(t[0] == 'm'):
        return 1
    else:
        return 0
 
train_df["Sex"] = train_df["Sex"].map(lambda a: isMale(a))
test_df["Sex"] = test_df["Sex"].map(lambda a: isMale(a))
train_df.head(3)



,Longevity,Education,Sex,Age,Pet,Children,Activity,Sport,Family,IsAlone,Age*Education,Status
0,0,3,0,73.0,1,0,725,1,1,0,6,3
1,1,1,1,75.0,1,0,7128,0,1,0,3,3
2,1,3,1,73.0,0,0,793,1,0,1,6,4


In [43]:
x_train = train_df[train_df.columns[1::]]
y_train = train_df['Longevity']

x_test = test_df[test_df.columns[1::]]
y_test = test_df['Longevity']

Так как цель предсказать Longevity, то делим данные.

In [44]:
x_train.head(10)

,Education,Sex,Age,Pet,Children,Activity,Sport,Family,IsAlone,Age*Education,Status
0,3,0,73.0,1,0,725,1,1,0,6,3
1,1,1,75.0,1,0,7128,0,1,0,3,3
2,3,1,73.0,0,0,793,1,0,1,6,4
3,1,1,74.0,1,0,5310,0,1,0,2,2
4,3,0,74.0,0,0,805,1,0,1,6,4
5,3,0,73.0,0,0,846,1,0,1,6,4
6,1,0,77.0,0,0,5186,0,0,1,4,5
7,3,0,70.0,3,1,2108,1,4,0,0,1
8,3,1,73.0,0,2,1113,1,2,0,6,3
9,2,1,72.0,1,0,3007,1,1,0,2,2


In [45]:
y_train.head(10)

0    0
1    1
2    1
3    1
4    0
5    0
6    0
7    0
8    1
9    1
Name: Longevity, dtype: int64

Случайный лес

Случайный лес — модель, состоящая из множества деревьев решений. Вместо того,чтобы просто усреднять прогнозы разных деревьев (такая концепция называется просто «лес»), эта модель использует две ключевые концепции, которые и делают этот лес случайным.

Случайная выборка образцов из набора данных при построении деревьев.
При разделении узлов выбираются случайные наборы параметров.

In [46]:
rf_classifier = ensemble.RandomForestClassifier(n_estimators = 50, max_depth = 10, random_state = 1)
#n_estimators - Количество деревьев в лесу.
#max_depth - глубина дерева
#random_state - это начальное число, используемое генератором случайных чисел;

In [47]:
rf_classifier.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=50, random_state=1)

In [48]:
y_pred = rf_classifier.predict(x_test)

Посчитаем ошибку с помощью mse.
Измеряет среднюю сумму квадратной разности между фактическим значением и прогнозируемым значением для всех точек данных. Выполняется возведение во вторую степень, поэтому отрицательные значения не компенсируют положительными. А также в силу свойств этой метрики, усиливается влияние ошибок, по квадратуре от исходного значения. Это значит, что если в в исходных измерениях мы ошиблись на 1, то метрика покажет 1, 2-4, 3-9 и так далее. Чем меньше MSE, тем точнее наше предсказание. Оптимум достигается в точке 0, то есть мы идеально предсказываем.

По сравнению с средней абсолютной ошибкой, MSE  имеет некоторые преимущества:
Она подчеркивает большие ошибки над меньших ошибках.
Является дифференцируемым, что позволяет более эффективно использовать для поиска минимальных или максимальных значений с помощью математических методов.

In [49]:
mse = metrics.mean_squared_error(y_test, y_pred) #квадратичное отклонение
mse

0.6084070796460177

In [50]:
ra = metrics.roc_auc_score(y_test, y_pred) #отношение правильно угаданных экземпляров класса 1 к неправильно угаданным экземплярам класса 0
# чем больше тем лучше
ra

0.5202093107181524

In [51]:
acc_random_forest = round(rf_classifier.score(x_train, y_train) * 100, 2)
acc_random_forest

92.88